<a href="https://colab.research.google.com/github/HuberAdrian/DataScience-Lectures/blob/main/GroupBy_Intro_week3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The ***GroupBy*** Pattern

We have seen the ***GroupBy*** operator in ***Pandas***, but  this is actually a more general ***design pattern*** that can be utilized in many data analyics frameworks and data access interfaces, e.g. in ***SQL***. 

## GroupBy: general Pattern
<img SRC="IMG/groupby.jpg">

#### GroupBy in SQL:
```
SELECT COUNT(CustomerID), Country
FROM Customers
GROUP BY Country
ORDER BY COUNT(CustomerID) DESC;
```

#### GroupBy in MongoDB
```
db.BusinessProcess.aggregate({
    "$group": {
        _id: {
            status: "$status",
            type: "$type"
        },
        count: {
            $sum: 1
        }
    }
   })
```


In [ ]:
#setup example
import numpy as np
import pandas as pd
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,1.221437,-1.517520
1,a,two,1.689427,1.246337
2,b,one,-0.953992,-0.657989
3,b,two,-0.470554,-2.072804
4,a,one,0.106501,-1.576961


In [ ]:
#group by key1 
grouped = df.groupby(df['key1'])
grouped #this is now a more complex group object

In [ ]:
#and generates a table per group
for name, group in grouped:
    print ("name:", name, "\n",group)

name: a 
   key1 key2     data1     data2
0    a  one  1.221437 -1.517520
1    a  two  1.689427  1.246337
4    a  one  0.106501 -1.576961
name: b 
   key1 key2     data1     data2
2    b  one -0.953992 -0.657989
3    b  two -0.470554 -2.072804


In [ ]:
#access group table
grouped.get_group('b')

,key1,key2,data1,data2
2,b,one,-0.953992,-0.657989
3,b,two,-0.470554,-2.072804


In [ ]:
#get numper of entries (rows) per group
grouped.size()

key1
a    3
b    2
dtype: int64

In [ ]:
#get number of group entries by columns
grouped.count()

,key2,data1,data2
key1,,,
a,3,3,3
b,2,2,2


#### Think of  grouped DataFrames as 3d objects:

In [ ]:
#accessing the "3d" group tables
grouped['data2'].get_group('a')

0   -1.517520
1    1.246337
4   -1.576961
Name: data2, dtype: float64

In [ ]:
grouped.get_group('a')['data2']

0   -1.517520
1    1.246337
4   -1.576961
Name: data2, dtype: float64

### Group by external keys

In [ ]:
#define external key years as numpy array
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([years]).mean()


2005    0.813437
2006   -0.423746
Name: data1, dtype: float64

### Group by functions

In [ ]:
#sort by column and retun top n
def top(df, n=5, column='data1'):
    return df.sort_values(by=column)[-n:]

df.groupby(df['key1']).apply(top,n=5)

key1 key2     data1     data2
key1                                
a    4    a  one  0.106501 -1.576961
     0    a  one  1.221437 -1.517520
     1    a  two  1.689427  1.246337
b    2    b  one -0.953992 -0.657989
     3    b  two -0.470554 -2.072804

### Group-wise aggregation (apply)
<img SRC="IMG/groupby.jpg">

#### Typical build in aggregation functions:
* sum
* mean
* max / min
* quantile
* ...

In [ ]:
#aggregate over the groups
grouped.count()

,key2,data1,data2
key1,,,
a,3,3,3
b,2,2,2


In [ ]:
grouped.sum()

,data1,data2
key1,,
a,3.017365,-1.848144
b,-1.424545,-2.730793


#### Custom Aggregation Functions

In [ ]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.582926,2.823297
b,0.483438,1.414815


#### Multiple aggregations

In [ ]:
#just call a list of function
grouped.agg([peak_to_peak, 'mean', 'median'])

data1                            data2                    
     peak_to_peak      mean    median peak_to_peak      mean    median
key1                                                                  
a        1.582926  1.005788  1.221437     2.823297 -0.616048 -1.517520
b        0.483438 -0.712273 -0.712273     1.414815 -1.365397 -1.365397

#### Suppressing the Group Keys 

In [ ]:
df.groupby(df['key1']).apply(top,n=2)

key1 key2     data1     data2
key1                                
a    0    a  one  1.221437 -1.517520
     1    a  two  1.689427  1.246337
b    2    b  one -0.953992 -0.657989
     3    b  two -0.470554 -2.072804

In [ ]:
df.groupby(df['key1'], group_keys=False).apply(top,n=2)

,key1,key2,data1,data2
0,a,one,1.221437,-1.517520
1,a,two,1.689427,1.246337
2,b,one,-0.953992,-0.657989
3,b,two,-0.470554,-2.072804


## More  Exercises in the Lab session... 